In [13]:
import gradio as gr
import pandas as pd
from huggingface_hub import hf_hub_download
import joblib

# Загрузка модели с Hugging Face
model_path = hf_hub_download(repo_id="Nikgorby/diplom_DS_SF", filename="random_forest_model (1).pkl")
model = joblib.load(model_path)

# Определение функции предсказания
def predict(csv_file):
    # Считывание CSV файла
    data = pd.read_csv(csv_file.name)
    # Предсказание
    predictions = model.predict(data)
    # Возвращение предсказаний в виде DataFrame
    result = predictions[0]
    # result = pd.DataFrame(predictions, columns=["Prediction"])
    return result

# Создание интерфейса Gradio
iface = gr.Interface(
    fn=predict,
    inputs=[gr.File(label="Upload CSV")],
    outputs=[gr.Textbox(label="Predictions")]
)

# Запуск приложения
iface.launch()


Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


In [ ]:
import gradio as gr

def generate_answer_git():
    # prepare question
    pass

def generate_answers(image, question):
    answer_git_base = generate_answer_git()
    return answer_git_base

outputs = [gr.Textbox(label="Answer generated by GIT-base")]
           
title = "Interactive gradio demo"
description = "Gradio Demo."
interface = gr.Interface(fn=generate_answers, 
                         inputs=[gr.DataFrame(), 
                                 gr.Textbox(label="Вопрос")],
                         outputs=outputs,
                         title=title,
                         description=description,
                         allow_flagging='never'
                         )
interface.launch()

In [ ]:
import gradio as gr


def filter_records(records, gender):
    return records[records["gender"] == gender]


demo = gr.Interface(
    filter_records,
    [
        gr.Dataframe(
            headers=["name", "age", "gender"],
            datatype=["str", "number", "str"],
            row_count=3,
            col_count=(3, "fixed"),
        ),
        gr.Dropdown(["M", "F", "O"]),
    ],
    "dataframe",
    description="Enter gender as 'M', 'F', or 'O' for other.",
)

if __name__ == "__main__":
    demo.launch()